### Example 4.2.1.1: 1-D interpolation on equally spaced input points

In [6]:
#Setting Peridynamic parameters
class PDoperator_:
    def __init__(self, number_input_points = None, xmin = None, xmax = None, 
               number_output_points = None, xvec_org = None, yvec_org = None, 
               n1order = None, nsize = None, order = None):
        self.totnode_org = number_input_points
        self.totnode = number_output_points
        self.xmin = xmin
        self.xmax = xmax
        self.xvec_org = xvec_org 
        self.yvec_org = yvec_org
        self.n1order = n1order
        self.nsize = nsize
        self.order = order
    

In [8]:
#Function that generates grid
class GridGenerator:
    def __init__(init, PDoperator):
        
        xmin = PDoperator.xmin 
        xmax = PDoperator.xmax
        totnode = PDoperator.totnode
        
        xvec = []
        dvolume = []
        dx = (xmax - xmin)/ totnode
        for i in range(totnode):
            xvec.append(round(xmin + dx*i +dx/2.0,6))
            dvolume.append(round(dx*1*1,6))
        init.xvec = xvec
        init.dvolume = dvolume
    
    

In [ ]:
#Function that creates lump volumes
def LumpVolumes(PDoperator, gridgenerator):
    
    totnode_org = PDoperator.totnode_org 
    totnode = PDoperator.totnode 
    xvec_org = PDoperator.xvec_org
    
    xvec = gridgenerator.xvec 
    dvolume = gridgenerator.dvolume
    
    
    dvolumes_org = [0] * totnode_org
    tiny = 1E-7
    rho = [0]*totnode_org
    dvolume_org = [0]*totnode_org
    for i in range(totnode):
        rhosum = 0.0
        for j in range(totnode_org):
            rho[j] = 1.0/((abs(xvec[i] - xvec_org[j]))**3 + tiny)
            rhosum = rhosum + rho[j]
        for j in range(totnode_org):
            dvolume_org[j] = dvolume_org[j] + (rho[j] / rhosum)*dvolume[i]
    rhosum = 0.0;
    for j in range(totnode_org):
        rhosum += dvolume_org[j]
    print("Total volume of output points         = ",totnode*dvolume[0] )
    print("Total volume of computed input points = ", rhosum)
    return dvolume_org

In [ ]:
#Calculates size of Peridynamic operator
def getSize1D(n1order):
    nsize = n1order + 1
    return nsize

In [ ]:
#Function that sets the order of the Differential Operators
def SetDiffOperators1D(nsize):
    order = list(range(0,nsize))
    return order

In [ ]:
class NodeFamily:
    def __init__(self, xsi_order = None, xsivec = None, deltax = None):
        self.xsi_order = xsi_order
        self.xsivec = xsivec
        self.deltax = deltax

In [ ]:
import pandas as pd
#Function that finds the node family for each Peridynamic node
def GenerateNodeFamily(xvec_org, xvec):
    
    xsi_order = pd.DataFrame({'node':xvec_org})
    xsivec = pd.DataFrame({'node':xvec_org})
    #xsivec = pd.DataFrame()
    
    #Ordering nodes
    for i in range(len(xvec)):
        aux = []
        for j in range(len(xvec_org)):
                aux.append(math.sqrt((xvec[i]-xvec_org[j])**2))
        xsi_order[str(xvec[i])] = aux
        xsivec[str(xvec[i])] = aux
   
    xsi_order.set_index('node', inplace=True)
    for i in xsi_order:
        xsi_order[str(i)] = xsi_order[str(i)].sort_values().keys()
    xsi_order_2 = pd.DataFrame({'node':xvec_org})
    for i in xsi_order:
        xsi_order_2[str(i)] = list(xsi_order[str(i)].sort_values().keys())
    
    #Calculating deltas (horizon) we are selecting the largest bond
    deltax = []
    xsivec = xsivec.drop(columns="node")
    for i in xsivec:
        deltax.append(max(list(xsivec[str(i)])))
    print(deltax)
    NodeFam = NodeFamily()
    xsi_order_2 = xsi_order_2.drop(columns="node")
    NodeFam.xsi_order = xsi_order_2
    NodeFam.xsivec = xsivec
    NodeFam.deltax = deltax
    
    #aux2 +=np.matmul(aux,np.transpose(aux))*dvolumes_org[node_mem]
    return NodeFam;

In [ ]:
#Function that calculates the Peridynamics weights
def calc_weights(NodeFam, xvec):
    tiny = 0.000001
    weights = pd.DataFrame() 

    k=0    
    for i in xvec:
        xsivec_column = NodeFam.xsivec[str(i)].to_numpy()
        aux = []
        for xsi_mag in xsivec_column:
            aux.append((NodeFam.deltax[k]/(xsi_mag+tiny))**2)
        weights[str(i)] = aux
        k = k + 1  
    return weights

In [ ]:
class p_operator():
    def __init__(self, p_vec=None, p_matrix = None, p_vec_prism = None):
        self.p_vec = p_vec
        self.p_matrix = p_matrix
        self.p_vec_prism = p_vec_prism
        

In [ ]:
import numpy as np
#Calculate p_operator
def p_operator_1d(x0,nsize,dvolumes_org, weights):
    
    x0_array = x0.to_numpy()
    weights_array = weights.to_numpy()
    aux2 = np.zeros((nsize,nsize))
    node_mem = 0;

    for i in x0_array:
        aux = np.zeros((nsize,1))
        for j in range(nsize):
            aux[j] = i**j
            print(aux[j])
          
        #aux2 +=np.matmul(aux,np.transpose(aux))
        #aux2 = aux2*dvolumes_org[node_mem]
        aux2 +=np.matmul(aux,np.transpose(aux))*dvolumes_org[node_mem]*weights[node_mem]
        #aux2 +=np.matmul(aux,np.transpose(aux))
        node_mem = node_mem + 1
    
    p_oper = p_operator()
    p_oper.p_vec = aux.reshape((1, nsize)) 
    p_oper. p_matrix = aux2
    return p_oper

In [ ]:
class DiffAmat():
    def __init__(self, p_vec_prism=None, A_prism = None, weights = None):
        self.p_vec_prism = p_vec_prism
        self.A_prism = A_prism
        self.weights = weights 

In [ ]:
#Form Diff A Matrix
import math

def FormDiffAmat1D(nsize, xvec_org, xvec, dvolumes_org, NodeFam):
    #An A matrix must be created for each of the Peridynamic nodes
    DiffAmat1D = [[0] * PDoperator.nsize] * PDoperator.nsize #Creating DiffAmat 2D matrix
    xsivec = pd.DataFrame()
    
    for i in range(len(xvec)):
        aux = []
        for j in range(len(xvec_org)):
                aux.append(xvec_org[j]-xvec[i])
        xsivec[str(xvec[i])] = aux
        
    #weights must be calculated here it is going to be one for each Peridynamic node i.e. 200
    weights = calc_weights(NodeFam, xvec)
   
    A_prism = np.zeros((len(xvec), nsize,nsize ))
    p_vec_prism = np.zeros((len(xvec), nsize))
    j=0
    for i in xvec:     
        p_operator = p_operator_1d(xsivec[str(i)],nsize, dvolumes_org,weights[str(i)])
        p_vec_prism[j,:] = p_operator.p_vec
        #print(p_operator.p_vec)
        A_prism[j,:,:] = p_operator.p_matrix
        j = j +1
    print(p_vec_prism)
    A_prism_info = DiffAmat()
    A_prism_info.p_vec_prism = p_vec_prism
    A_prism_info.A_prism = A_prism
    A_prism_info.weights = weights
    return A_prism_info
    

In [ ]:
def FormInvDiffAmat1D(DiffAmat1D, nsize):
    a,b,c = DiffAmat1D.shape #get dimensions of DiffAmat1D
    DiffAmatInv1D = np.zeros((a, nsize,nsize ))
    for i in range(a):
        DiffAmatInv1D[i] = np.linalg.inv(DiffAmat1D[i])

    return DiffAmatInv1D

In [ ]:
def FormDiffAvec(DiffAmatInv1D,DiffBvec1D):
    a,b,c = DiffAmatInv1D.shape #get dimensions of DiffAmatInv1D
    DiffAvec_prism = np.zeros((a, b))

    for i in range(a): 
        DiffAvec_prism[i,:] = np.matmul(DiffBvec1D,DiffAmatInv1D[i])
    return DiffAvec_prism

In [ ]:
def gfun(DiffAvec, plist, weight):
    

In [ ]:
import numpy as np
import math
#Function that calculates the 1D pass derivative operator
def Derivative_Operator_1D_PASS(PDoperator, gridgenerator, dvolumes_org):
    
    xvec = gridgenerator.xvec
    n1order = PDoperator.n1order
    PDoperator.nsize = getSize1D(n1order)
    PDoperator.order = SetDiffOperators1D(PDoperator.nsize)
    DiffAmat1D = [[0] * PDoperator.nsize] * PDoperator.nsize #Creating DiffAmat 2D matrix
    DiffAmatInv1D = [[0] * PDoperator.nsize] * PDoperator.nsize #Creating DiffAmatInv 2D matrix
    DiffBvec1D = [0] * PDoperator.nsize
    DiffAvec1D = [0] * PDoperator.nsize
    plist = [0] * PDoperator.nsize
    blist = [0] * PDoperator.nsize
    weight = [0] * PDoperator.nsize
    rcvec = [0] * PDoperator.nsize
    wgt = [0] * PDoperator.totnode_org
    ndfam = [0] * PDoperator.totnode_org
    
   
    nsize = PDoperator.nsize
    
    xvec_org = PDoperator.xvec_org
    #Here is where we should calculate A matrix, inverse of A matrix and B vector; for every Peridynamic node.
    
    
    #Generate node family
    NodeFam = GenerateNodeFamily(xvec_org, xvec)
    #Form A array of A Matrices and p_vec_prism
    DiffAmat1D = FormDiffAmat1D(nsize, xvec_org, xvec, dvolumes_org, NodeFam);    
    #Form Inverse of A Matrices
    DiffAmatInv1D = FormInvDiffAmat1D(DiffAmat1D.A_prism, nsize)
    #Form B vector
    DiffBvec1D = np.array([1,0,0])
    #Form a vector
    DiffAvec = FormDiffAvec(DiffAmatInv1D, DiffBvec1D)
    #gfunval
    
    #print(DiffAvec[0])
    #print(DiffAmat1D.p_vec_prism[0])
    #h = DiffAmat1D.weights['0.0225'].to_numpy()
    #print(h[0])
    
    #print(h[0]*np.matmul(DiffAvec[0],np.transpose(DiffAmat1D.p_vec_prism[0])))
    #print(DiffAmat1D.weights['0.0225'].to_numpy())
    
    #print("Interpolating for N=2")
    #for i in range(PDoperator.totnode):
        #GenerateNodeFamily_0
        
    
    return DiffAvec
    
    

In [ ]:
#The PDoperator class constructor has the following inputs
#PDoperator_name = PDoperator(number_input_points, xmin, xmax, output_points, 
#                             xvec_org, yvec_org)
number_input_points = 10
xmin = 0
xmax = 9
number_output_points = 200
xvec_org = list(range(xmin,xmax+1))
yvec_org = [2, 2, 2, 2, 2, 3.2, 3.2, 3.2, 3.2, 3.2]
n1order = 2

PDoperator = PDoperator_(number_input_points, xmin, xmax, number_output_points, 
                        xvec_org, yvec_org, n1order)

#Generating the grid
gridgenerator = GridGenerator(PDoperator)

#Create lump volumes
dvolumes_org = LumpVolumes(PDoperator, gridgenerator)
#Calculating the 1D derivative operator
h = Derivative_Operator_1D_PASS(PDoperator, gridgenerator,dvolumes_org)



In [ ]:
print(h)

In [ ]:
#h.to_excel("output.xlsx")

In [ ]:

DiffBvec1D = np.array([1,0,0]) 
print(h)
#print(np.matmul(DiffBvec1D,h[1]))

In [ ]:
print(gridgenerator.xvec)
print(xvec_org[2]-gridgenerator.xvec[0])

In [37]:
import math
#In this class we create a pandas dataframe that hold the values for Xsi and their magnitudes
class FormXsiDataFrame:
    def __init__(init, xvec_org, xvec):
        xsivec = pd.DataFrame({'node':xvec_org})
        xsivec_mags = pd.DataFrame({'node':xvec_org})
    
        for i in range(len(xvec)):
            aux = []
            aux2 = []
            for j in range(len(xvec_org)):
                aux.append(xvec_org[j]-xvec[i])
                aux2.append(math.sqrt(((xvec_org[j]-xvec[i])**2)))
            xsivec[str(xvec[i])] = aux
            xsivec_mags[str(xvec[i])] = aux2
        
        init.xsivec = xsivec
        init.xsivec_mags = xsivec_mags


In [26]:
#This class creates a list of all p lists
class FormPlistPrism:
    def __init__(init, xsivec_matrix):
        totnode_org, totnode =  xsivec_matrix.shape
        print(xsivec_matrix.shape)
        p_list = np.zeros((totnode_org, totnode, 3)) #The 3 is hard coded which should be change later
        node_count = 0
        for node in xsivec_matrix:
            xsivec = xsivec_matrix[str(node)]
            xsi_count = 0
            for xsi in xsivec:
                aux = np.zeros((3,1))
                for j in range(3):
                    aux[j] = xsi**j                
                p_list[xsi_count, node_count, :] = aux.reshape((1, 3))
                xsi_count = xsi_count + 1
            node_count = node_count + 1
        
        init.p_list = p_list


In [3]:
import pandas as pd
#Function that finds the node family for each Peridynamic node
class GenerateNodeFamily:
    def __init__(init, xsi_df):
    
        xsivec_mags = xsi_df.xsivec_mags.copy()
        xsi_order = xsi_df.xsivec_mags.copy()
   
        xsi_order.set_index('node', inplace=True)    
        for i in xsi_order:
            xsi_order[str(i)] = xsi_order[str(i)].sort_values().keys()
        
        xsi_order_2 = pd.DataFrame({'node':xvec_org})
        for i in xsi_order:
            xsi_order_2[str(i)] = list(xsi_order[str(i)].sort_values().keys())
    
        #Calculating deltas (horizon) we are selecting the largest bond
        deltax = []    
        xsivec_mags = xsivec_mags.drop(columns="node")
        for i in xsivec_mags:
            deltax.append(max(list(xsivec_mags[str(i)])))
    
        init.xsi_order = xsi_order_2.drop(columns="node")
        init.deltax = deltax


In [10]:
#Function that creates lump volumes
class LumpVolumes:
    def __init__(init, totnode_org, totnode, xvec_org, xvec, dvolume):           
    
        dvolumes_org = [0] * totnode_org
        tiny = 1E-7
        rho = [0]*totnode_org
        dvolume_org = [0]*totnode_org
        for i in range(totnode):
            rhosum = 0.0
            for j in range(totnode_org):
                rho[j] = 1.0/((abs(xvec[i] - xvec_org[j]))**3 + tiny)
                rhosum = rhosum + rho[j]
            for j in range(totnode_org):
                dvolume_org[j] = dvolume_org[j] + (rho[j] / rhosum)*dvolume[i]
        rhosum = 0.0;
        for j in range(totnode_org):
            rhosum += dvolume_org[j]
        print("Total volume of output points         = ",totnode*dvolume[0] )
        print("Total volume of computed input points = ", rhosum)
        
        init.dvolume_org = dvolume_org

In [17]:
#Function that calculates the Peridynamics weights
class FormWeightDataFrame:
    def __init__(init, xsivec, deltax, xvec):
        tiny = 0.000001
        weights = pd.DataFrame() 
        k=0    
        for i in xvec:
            xsivec_column = xsivec[str(i)].to_numpy()
            aux = []
            for xsi in xsivec_column:
                aux.append((deltax[k]/(xsi+tiny))**2)
            weights[str(i)] = aux
            k = k + 1  
        init.weights = weights

In [ ]:
import numpy as np
#Calculate p_operator
def p_operator_1d(x0,nsize,dvolumes_org, weights):
    
    x0_array = x0.to_numpy()
    weights_array = weights.to_numpy()
    aux2 = np.zeros((nsize,nsize))
    node_mem = 0;

    for i in x0_array:
        aux = np.zeros((nsize,1))
        for j in range(nsize):
            aux[j] = i**j
            print(aux[j])
          
        #aux2 +=np.matmul(aux,np.transpose(aux))
        #aux2 = aux2*dvolumes_org[node_mem]
        aux2 +=np.matmul(aux,np.transpose(aux))*dvolumes_org[node_mem]*weights[node_mem]
        #aux2 +=np.matmul(aux,np.transpose(aux))
        node_mem = node_mem + 1
    
    p_oper = p_operator()
    p_oper.p_vec = aux.reshape((1, nsize)) 
    p_oper. p_matrix = aux2
    return p_oper

In [ ]:
#Form Diff A Matrix
import math

class FormDiffAmat1D:
    def __init__(nsize, xvec_org, xvec, dvolumes_org, NodeFam):
        #An A matrix must be created for each of the Peridynamic nodes
        DiffAmat1D = [[0] * PDoperator.nsize] * PDoperator.nsize #Creating DiffAmat 2D matrix
        xsivec = pd.DataFrame()

        for i in range(len(xvec)):
            aux = []
            for j in range(len(xvec_org)):
                    aux.append(xvec_org[j]-xvec[i])
            xsivec[str(xvec[i])] = aux

        #weights must be calculated here it is going to be one for each Peridynamic node i.e. 200
        weights = calc_weights(NodeFam, xvec)

        A_prism = np.zeros((len(xvec), nsize,nsize ))
        p_vec_prism = np.zeros((len(xvec), nsize))
        j=0
        for i in xvec:     
            p_operator = p_operator_1d(xsivec[str(i)],nsize, dvolumes_org,weights[str(i)])
            p_vec_prism[j,:] = p_operator.p_vec
            #print(p_operator.p_vec)
            A_prism[j,:,:] = p_operator.p_matrix
            j = j +1
        print(p_vec_prism)
        A_prism_info = DiffAmat()
        A_prism_info.p_vec_prism = p_vec_prism
        A_prism_info.A_prism = A_prism
        A_prism_info.weights = weights
        return A_prism_info

In [38]:
import numpy as np
number_input_points = 10
xmin = 0
xmax = 9
number_output_points = 200
xvec_org = list(range(xmin,xmax+1))
yvec_org = [2, 2, 2, 2, 2, 3.2, 3.2, 3.2, 3.2, 3.2]
n1order = 2

PDoperator = PDoperator_(number_input_points, xmin, xmax, number_output_points, 
                        xvec_org, yvec_org, n1order)

#Generating the grid
gridgenerator = GridGenerator(PDoperator)

#Create xsi dataframe
xsi_df = FormXsiDataFrame(PDoperator.xvec_org, gridgenerator.xvec)

#Create plist 
p_list = FormPlistPrism(xsi_df.xsivec)

#Generate node family
NodeFam = GenerateNodeFamily(xsi_df)

#Create lump volumes
dvolumes_org = LumpVolumes(PDoperator.totnode_org, PDoperator.totnode, PDoperator.xvec_org, gridgenerator.xvec, 
                           gridgenerator.dvolume)

#Calculating Peridynamic weights
weights = FormWeightDataFrame(xsi_df.xsivec, NodeFam.deltax, gridgenerator.xvec)

#Form A array of A Matrices and p_vec_prism
#DiffAmat1D = FormDiffAmat1D(PDoperator.nsize, PDoperator.xvec_org, gridgenerator.xvec, dvolumes_org.dvolumes_org, 
#                            NodeFam);

print(xsi_df.xsivec)
print(weights.weights.shape)
print(p_list.p_list.shape)
#print(dvolumes_org.dvolume_org)



(10, 201)
Total volume of output points         =  9.0
Total volume of computed input points =  9.0
   node  0.0225  0.0675  0.1125  0.1575  0.2025  0.2475  0.2925  0.3375  \
0     0 -0.0225 -0.0675 -0.1125 -0.1575 -0.2025 -0.2475 -0.2925 -0.3375   
1     1  0.9775  0.9325  0.8875  0.8425  0.7975  0.7525  0.7075  0.6625   
2     2  1.9775  1.9325  1.8875  1.8425  1.7975  1.7525  1.7075  1.6625   
3     3  2.9775  2.9325  2.8875  2.8425  2.7975  2.7525  2.7075  2.6625   
4     4  3.9775  3.9325  3.8875  3.8425  3.7975  3.7525  3.7075  3.6625   
5     5  4.9775  4.9325  4.8875  4.8425  4.7975  4.7525  4.7075  4.6625   
6     6  5.9775  5.9325  5.8875  5.8425  5.7975  5.7525  5.7075  5.6625   
7     7  6.9775  6.9325  6.8875  6.8425  6.7975  6.7525  6.7075  6.6625   
8     8  7.9775  7.9325  7.8875  7.8425  7.7975  7.7525  7.7075  7.6625   
9     9  8.9775  8.9325  8.8875  8.8425  8.7975  8.7525  8.7075  8.6625   

   0.3825  ...  8.5725  8.6175  8.6625  8.7075  8.7525  8.7975  8.8425  \
